In [ ]:
from latex2sympy2 import latex2sympy
memory = {'问题':[],'数据集答案':[],'给出AI答案边的数据集答案':[],'AI答案':[],'判别正误[1/0]':[]}
# num 类
# count = 0
for i in range(len(dataset1['train'])):
    if i > 1999: break
    []
    memory['问题'].append(dataset1['train'][i]['text'])
    memory['数据集答案'].append(dataset1['train'][i]['answer_expressions'])
    memory['给出AI答案边的数据集答案'].append(res[i][1])
    memory['AI答案'].append(res[i][2])
    
    if i in all_label['num_label']:
        # count += 1
        item = dataset1['train'][i]
        print('-'*30)
        print('序列号标答', item['answer_expressions'])
        result = numexpr.evaluate(item['answer_expressions'])
        print('float', result)

        print('GPT答案', res[i][2]) 

        res[i][2] = re.sub(r'√(\d+)', r'\\sqrt{\1}', res[i][2])
        if '$' not in res[i][2]:
            res[i][2] = '$'+res[i][2]+'$'
        answer_now = re.findall(r'\$(.*?)\$', res[i][2])
        answer_now = [re.sub(r'[\u4e00-\u9fff\uff00-\uffef，。]', '', x) for x in answer_now] # 去掉汉字
        answer_now = [re.sub(r'^.*=\s*', '', x).replace(' ', '') for x in answer_now] # 去掉等号前面的东西 
        answer_now = [x for x in answer_now if len(x)>0] # 去掉空字符串

        modified_answer_now = []
        for item_now in answer_now:
            if not check_string(item_now):
                # 去掉 frac sqrt 之后没有字母了
                modified_answer_now.append(item_now)
        answer_now = modified_answer_now
            
        if len(answer_now) < 1:
            print('错误答案-'+'-'*30)
            memory['判别正误[1/0]'].append(0)
            continue
        if len(answer_now) > 1:
            print('人工判断1'+'-'*30)
            memory['判别正误[1/0]'].append(100)
            continue

        # if check_string(answer_now[0]):
        #     # 去掉 frac sqrt 仍有字母 回答错误
        #     print('float GPT answer', res[i][2])
        # elif pattern.search(answer_now[0]):
        #     # 有中文
        #     print('人工判断-'+res[i][2]+'-'*30)
            
        else:
            try:
                sympy_expr = latex2sympy(answer_now[0])
                float_value = float(sympy_expr.evalf())
                print('float GPT answer', float_value)
                if result == float_value:
                    print('正确答案-'+'-'*30)
                    memory['判别正误[1/0]'].append(1)
                else:
                    print('错误答案-'+'-'*30)
                    memory['判别正误[1/0]'].append(0)
            except TypeError:
                try:
                    float_value = numexpr.evaluate(answer_now[0])
                    print('float GPT answer', float_value)
                    if result == float_value:
                        print('正确答案-'+'-'*30)
                        memory['判别正误[1/0]'].append(1)
                    else:
                        print('错误答案-'+'-'*30)
                        memory['判别正误[1/0]'].append(0)
                except (SyntaxError, Exception):
                    print('人工判断2-'+'-'*30)
                    memory['判别正误[1/0]'].append(100)
            except (SyntaxError, Exception):
                print('人工判断2-'+'-'*30)
                memory['判别正误[1/0]'].append(100)
    else:
        memory['判别正误[1/0]'].append(100)


In [ ]:
import pandas as pd
# 需要库：openpyxl
# 将字典转换为pandas DataFrame
memory_df = pd.DataFrame(memory)
 
# 将DataFrame写入Excel文件
# !!!!记得修改你的文件名
memory_df.to_excel('qwen2.xlsx', index=False)